In [1]:
import pandas as pd
from datasets import Dataset


train_prompts_df = pd.read_json('../fn1.7-train-prompts.jsonl', lines=True)
train_prompts = Dataset.from_pandas(train_prompts_df)


/home/jacob/miniconda3/envs/fsp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 5020
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="llama-3.2-3b-fsp-ft",
    model_name="unsloth/Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    dtype=None,
)

==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 SUPER. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
Unsloth 2024.11.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

def formatting_prompts_func_test(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo[:-1], tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = train_prompts.map(formatting_prompts_func, batched = True,)

test_prompts_df = pd.read_json('../fn1.7-test-prompts.jsonl', lines=True)
test_prompts = Dataset.from_pandas(test_prompts_df)

test_dataset = test_prompts.map(formatting_prompts_func_test, batched = True,)

Map: 100%|██████████| 6223/6223 [00:00<00:00, 22527.41 examples/s]


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 20,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2): 100%|██████████| 18865/18865 [00:11<00:00, 1686.52 examples/s]


In [6]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/18865 [00:00<?, ? examples/s]

Map: 100%|██████████| 18865/18865 [00:05<00:00, 3437.10 examples/s]


In [7]:
import torch
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4070 SUPER. Max memory = 11.994 GB.
2.725 GB of memory reserved.


In [8]:
# trainer_stats = trainer.train()
trainer_stats = trainer.train(resume_from_checkpoint='outputs/checkpoint-2358')

/home/jacob/miniconda3/envs/fsp/lib/python3.12/site-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, 

Step,Training Loss
2359,0.054400
2360,0.049700
2361,0.093900
2362,0.028400
2363,0.038900
2364,0.034700
2365,0.029600
2366,0.043900
2367,0.075100
2368,0.040200


In [7]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

predictions = []

for i, example in enumerate(test_dataset['messages']):
    inputs = tokenizer.apply_chat_template(
        example[:-1],
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
    # Decode the generated text, add to predictions, ignore prompt in output
    predictions.append(tokenizer.decode(outputs[0]).split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1])

test_prompts_df['responses'] = predictions
test_prompts_df.to_json('llama-3.2-3B-fsp-2ep-preds.jsonl', orient='records', lines=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [10]:
model.save_pretrained("llama-3.2-3b-fsp-ft2") # Local saving
tokenizer.save_pretrained("llama-3.2-3b-fsp-ft2")

('llama-3.2-3b-fsp-ft2/tokenizer_config.json',
 'llama-3.2-3b-fsp-ft2/special_tokens_map.json',
 'llama-3.2-3b-fsp-ft2/tokenizer.json')

In [22]:
x = test_prompts_df.head(len(predictions)).copy()
x['prediction'] = predictions
x.to_json('half-pred.jsonl', lines=True, orient='records')